# Building Autonomous SQL Assistant Agents with Strands Agents SDK

This notebook demonstrates two approaches to building AI agents with Strands Agents:

1. **Schema-Guided Agents** - Agents with pre-defined data schema knowledge
2. **Autonomous Discovery Agents** - Agents that discover and explore data schemas independently

We'll compare vanilla vs fine-tuned models in both scenarios to understand the impact of domain-specific training.

In [ ]:
%store -r VANILLA_ENDPOINT_NAME CUSTOM_ENDPOINT_NAME

test_query = "Which 5 countries have performed the most returns of Technology products bought with Same Day shipping?"

## Part 1: Schema-Guided Autonomous Agents

In this approach, we provide the agent with detailed knowledge of our data schema upfront. This simulates scenarios where you have well-documented data catalogs and want agents to work within known structures.

In [ ]:
data_schema = open("utils/data_schema.md").read()
schema_guided_prompt = f"""Act like an AWS Data Analyst. Help solve queries using data catalog.
Steps: 1) Generate SQL 2) Execute query 3) Return results

{data_schema}
"""

In [ ]:
from strands.models.sagemaker import SageMakerAIModel

payload_config = {"max_tokens": 1024*4, "temperature": 0, "stream": True}

vanilla_model = SageMakerAIModel(
    endpoint_config={"endpoint_name": VANILLA_ENDPOINT_NAME},
    payload_config=payload_config
)

custom_model = SageMakerAIModel(
    endpoint_config={"endpoint_name": CUSTOM_ENDPOINT_NAME},
    payload_config=payload_config
)

In [ ]:
from utils.custom_tools import *
import os

os.environ["BYPASS_TOOL_CONSENT"] = "true"
os.environ["STRANDS_TOOL_CONSOLE_MODE"] = "true"

schema_guided_tools = [execute_query, get_query_results]
discovery_tools = [execute_query, get_query_results, get_samples, get_tables, list_databases]

### Schema-Guided Agent Execution

Let's create and test both vanilla and custom agents with pre-defined schema knowledge.

In [ ]:
from strands import Agent

print("=== SCHEMA-GUIDED: VANILLA MODEL ===")
vanilla_schema_agent = Agent(vanilla_model, tools=schema_guided_tools, system_prompt=schema_guided_prompt)
vanilla_response = vanilla_schema_agent(test_query)

In [ ]:
print("=== SCHEMA-GUIDED: FINE-TUNED MODEL ===")
custom_schema_agent = Agent(custom_model, tools=schema_guided_tools, system_prompt=schema_guided_prompt)
custom_response = custom_schema_agent(test_query)

## Part 2: Autonomous Discovery Agents

In this advanced approach, agents start with no schema knowledge and must discover, explore, and understand the data structure autonomously. This demonstrates true autonomous intelligence and adaptability.

In [ ]:
discovery_prompt = """Act like an AWS Data Analyst. Help solve queries using data catalog discovery.
Steps: 1) Discover databases/tables 2) Sample data 3) Generate SQL 4) Execute 5) Return results
"""

In [ ]:
from utils.custom_tools import *

discovery_tools = [execute_query, get_query_results, get_samples, get_tables, list_databases]

### Autonomous Discovery Agent Execution

Watch how these agents autonomously discover the data structure and build understanding from scratch.

In [ ]:
print("=== AUTONOMOUS DISCOVERY: VANILLA MODEL ===")
vanilla_discovery_agent = Agent(vanilla_model, tools=discovery_tools, system_prompt=discovery_prompt)
vanilla_discovery_result = vanilla_discovery_agent(test_query)

In [ ]:
print("=== AUTONOMOUS DISCOVERY: FINE-TUNED MODEL ===")
custom_discovery_agent = Agent(custom_model, tools=discovery_tools, system_prompt=discovery_prompt)
custom_discovery_result = custom_discovery_agent(test_query)

## Key Findings

**Schema-Guided vs Autonomous Discovery:**
- Schema-guided: Faster with pre-defined knowledge
- Discovery: Demonstrates adaptability and exploration
- Both benefit from fine-tuning

**Vanilla vs Fine-Tuned Models:**
- Fine-tuned models show superior SQL accuracy
- Better business terminology understanding
- More efficient tool usage

**Business Applications:**
- Schema-guided: Stable, documented environments
- Discovery: Dynamic, ad-hoc analysis
- Fine-tuning: Significant accuracy improvements